<a href="https://www.kaggle.com/code/sridharstreaks/ml-model-overfitting-the-data?scriptVersionId=121540398" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# EDA and Creating model to Predicts Maths Score.

#### In this notebook i tried to create a Ml model that predicts the "Maths Score" using various parameters.

**I tried using linear regression and random forest but those end up overfitting results. I'll futher dicuss why that happened and discuss measures to mitigate the issues at the end of the notebook**

#### Lets's Begin

# Loading Libraries and Dataset

In [1]:
import pandas as pd
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [2]:
df=pd.read_csv("/kaggle/input/student-performance-in-mathematics/Student performance in mathematics.csv")

In [3]:
df.head(10)

,Student ID,Gender,Race/ethnicity,Parental level of education,Lunch,Test preparation course,Math score,Reading score,Writing score
0,1,Female,Group B,Bachelor's degree,Yes,Completed,72,72,74
1,2,Female,Group C,Some college,No,None,69,90,88
2,3,Female,Group B,Master's degree,No,Completed,90,95,93
3,4,Male,Group A,Associate's degree,Yes,None,47,57,44
4,5,Male,Group C,Some college,Yes,None,76,78,75
5,6,Female,Group B,Associate's degree,No,Completed,71,83,78
6,7,Female,Group B,Some college,No,None,88,95,92
7,8,Male,Group B,Some college,Yes,None,40,43,39
8,9,Male,Group D,High school,Yes,Completed,64,64,67
9,10,Female,Group B,High school,Yes,None,38,60,50


--------------------------------------------------------------------------------------------------------

# Overviewing the data

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Student ID                   207 non-null    int64 
 1   Gender                       207 non-null    object
 2   Race/ethnicity               207 non-null    object
 3   Parental level of education  207 non-null    object
 4   Lunch                        207 non-null    object
 5   Test preparation course      207 non-null    object
 6   Math score                   207 non-null    int64 
 7   Reading score                207 non-null    int64 
 8   Writing score                207 non-null    int64 
dtypes: int64(4), object(5)
memory usage: 14.7+ KB


In [5]:
df.describe()

,Student ID,Math score,Reading score,Writing score
count,207.000000,207.000000,207.000000,207.000000
mean,104.000000,69.207729,69.898551,68.971014
std,59.899917,13.528838,14.960607,15.843259
min,1.000000,38.000000,38.000000,35.000000
25%,52.500000,59.500000,59.000000,57.000000
50%,104.000000,69.000000,70.000000,70.000000
75%,155.500000,79.000000,80.000000,80.000000
max,207.000000,98.000000,100.000000,100.000000


In [6]:
df.isna().sum()

Student ID                     0
Gender                         0
Race/ethnicity                 0
Parental level of education    0
Lunch                          0
Test preparation course        0
Math score                     0
Reading score                  0
Writing score                  0
dtype: int64

--------------------------------------------------------------------------------------------------------

# EDA Through Plots

In [7]:
import plotly.express as px

fig = px.scatter(df, x='Math score', y='Reading score', 
                 color='Gender',title='Maths Scores vs Reading Scores')
fig.show()

In [8]:
fig = px.box(df, x='Race/ethnicity', y='Writing score', 
             color='Test preparation course',title='Ethnicity vs Writing scores')
fig.show()

In [9]:
fig = px.histogram(df, x='Math score', nbins=30, 
                   color='Parental level of education',title='Math Score Distribution')
fig.show()

In [10]:
fig = px.bar(df, x='Race/ethnicity', y='Math score', color='Gender',title='Math Score by Ethinicity')
fig.show()

In [11]:
fig = px.scatter_matrix(df, dimensions=['Math score', 'Reading score', 'Writing score'], 
                        color='Race/ethnicity',title='Scatter Matrix between Math, Reading and Writing Scores')
fig.show()

In [12]:
fig = px.violin(df, x='Test preparation course', y='Reading score', 
                color='Parental level of education', box=True, points='all',
                title='Test preparation course vs Reading score')
fig.show()

--------------------------------------------------------------------------------------------------------

# Data preprocessing For ML Modelling

**I'm using Lablel Encoder to change catogory columns to numeric equalent for regression**

In [13]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Preprocess the data
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])
df['Race/ethnicity'] = le.fit_transform(df['Race/ethnicity'])
df['Parental level of education'] = le.fit_transform(df['Parental level of education'])
df['Lunch'] = le.fit_transform(df['Lunch'])
df['Test preparation course'] = le.fit_transform(df['Test preparation course'])

scaler = StandardScaler()
df[['Reading score', 'Writing score']] = scaler.fit_transform(df[['Reading score', 'Writing score']])

In [14]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X = df.drop(['Math score', 'Reading score', 'Writing score'], axis=1)
y = df['Math score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Implementing Regression Models on Pre-Processed Data

In [15]:
from sklearn.linear_model import LinearRegression

# Train a linear regression model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Evaluate the model
train_score = lr.score(X_train, y_train)
test_score = lr.score(X_test, y_test)

print('Training score:', train_score)
print('Testing score:', test_score)

Training score: 0.3832909552981765
Testing score: 0.5121263478667983


**The model has a training score of 0.383 and a testing score of 0.512. This suggests that the model may be overfitting to the training data, and may not generalize well to new data. we may want to try using a more complex model, or using a regularization technique such as Lasso or Ridge Regression to improve the performance of the model. Additionally, we can also try to engineer new features or use feature selection techniques to improve the model's performance.**

**So, I tried the slightly coplex Random Forest Regressor**

In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Train a Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Evaluate the model
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

train_score = r2_score(y_train, y_train_pred)
test_score = r2_score(y_test, y_test_pred)

print('Training score:', train_score)
print('Testing score:', test_score)

Training score: 0.8951748547727831
Testing score: 0.36044122008952106


**The model has a training score of 0.895 and a testing score of 0.360. Again, This suggests that the model is overfitting to the training data, and is not able to generalize well to new data. One possible reason for this could be that the model is too complex, and is fitting the noise in the training data.**

**So, This time i tried using other ensemble methods such as Gradient Boosting or XGBoost to improve the model's performance.**

In [17]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import xgboost as xgb

# Train a Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42, min_samples_split=10, max_depth=10)
rf.fit(X_train, y_train)

# Train a Gradient Boosting Regressor
gb = GradientBoostingRegressor(n_estimators=100, random_state=42, learning_rate=0.1, max_depth=3)
gb.fit(X_train, y_train)

# Train an XGBoost Regressor
xg = xgb.XGBRegressor(n_estimators=100, random_state=42, learning_rate=0.1, max_depth=3)
xg.fit(X_train, y_train)

# Evaluate the models
rf_train_score = rf.score(X_train, y_train)
rf_test_score = rf.score(X_test, y_test)
gb_train_score = gb.score(X_train, y_train)
gb_test_score = gb.score(X_test, y_test)
xg_train_score = xg.score(X_train, y_train)
xg_test_score = xg.score(X_test, y_test)

print('Random Forest - Training score:', rf_train_score)
print('Random Forest - Testing score:', rf_test_score)
print('Gradient Boosting - Training score:', gb_train_score)
print('Gradient Boosting - Testing score:', gb_test_score)
print('XGBoost - Training score:', xg_train_score)
print('XGBoost - Testing score:', xg_test_score)

Random Forest - Training score: 0.6923764745094994
Random Forest - Testing score: 0.4338522114147687
Gradient Boosting - Training score: 0.8408366448401194
Gradient Boosting - Testing score: 0.525739990198503
XGBoost - Training score: 0.7892983253269024
XGBoost - Testing score: 0.458035739693027


**It looks like the gradient boosting model had the best performance among the ensemble methods with a testing score of 0.5257. The random forest and XGBoost models had lower testing scores, but it's still worth considering their performance as they may perform better on different datasets.**

**Lastly, i tried "Hyperparameter tuning" hyperparameter tuning is a good way to try and improve the performance of your model. One common way to perform hyperparameter tuning is by using a grid search, which tests a range of hyperparameters and finds the combination that performs the best.**

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Define the model
rf_model = RandomForestRegressor(random_state=42)

# Define the grid of hyperparameters to search
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20],
    'max_features': ['sqrt', 'log2']
}

# Define the grid search object
grid_search = GridSearchCV(rf_model, param_grid, cv=5)

# Fit the grid search object to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

# Evaluate the model with best parameters on the test data
best_rf_model = grid_search.best_estimator_
test_score = best_rf_model.score(X_test, y_test)
print("Test score with best parameters:", test_score)


Best parameters: {'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 100}
Best score: 0.29386185781180474
Test score with best parameters: 0.42791612433516435


**The best score achieved by the model with these hyperparameters on the training data is 0.2938, which is higher than the training score reported earlier without hyperparameter tuning. However, the testing score with the best parameters is 0.4279, which is lower than the testing score reported earlier without hyperparameter tuning.**

**This suggests that the hyperparameter tuning has improved the performance of the model on the training data, but it did not generalize well to the test data.**

# The problem And Final Thoughts

***From here on the model will get complexing but we might not get better results and here why***

***As far as i know he size of the dataset could be a limiting factor in achieving good results and can increase the risk of overfitting. With a small dataset, it may be difficult to capture the complex patterns and relationships between the features and the target variable accurately.***

***Overfitting occurs when a model learns the noise or random fluctuations in the training data instead of the underlying patterns. This can happen when the model is too complex and has too many parameters relative to the size of the training data.***

**Hope this notebook gave a undertanding of overfitting data even though the models aren't accurate**

# The End